<a href="https://colab.research.google.com/github/braydencjr/GoalGuru/blob/main/GoalGuru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##COMPLETE SMART CAREER ADVISOR - ENHANCED WITH DUAL SEARCH & USER MEMORY

Install required packages

In [ ]:
!pip install -q langchain langchain-google-genai gradio googlesearch-python
#Install Tavily for enhanced search results
!pip install -q tavily-python

import os, getpass
import json
import re
from typing import Dict, List, Any, Optional
from datetime import datetime
import gradio as gr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


SET UP API KEYS

In [ ]:
# Setup Gemini API Key
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("🔑 Enter your Gemini API key:")

# Setup Tavily API Key (for better search results)
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("🔑 Enter your TAVILY API key:")

# Import libraries with error handling
try:
    from langchain_google_genai import ChatGoogleGenerativeAI
    from langchain_core.messages import HumanMessage

    llm_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        convert_system_message_to_human=True
    )

    from langchain.schema import HumanMessage
    LLM_AVAILABLE = True

except ImportError:
    print("⚠️ LangChain not available")
    LLM_AVAILABLE = False

try:
    from googlesearch import search
    GOOGLE_SEARCH_AVAILABLE = True
except ImportError:
    print("⚠️ Google search not available")
    GOOGLE_SEARCH_AVAILABLE = False

try:
    from tavily import TavilyClient
    TAVILY_AVAILABLE = bool(os.environ.get("TAVILY_API_KEY", "").strip())
    if TAVILY_AVAILABLE:
        print("✅ Tavily search available")
except ImportError:
    TAVILY_AVAILABLE = False
    print("💡 Install Tavily for enhanced search: pip install tavily-python")

🔑 Enter your Gemini API key:··········
🔑 Enter your TAVILY API key:··········
✅ Tavily search available


SET UP MEMORY SYSTEM WITH COMPLETE CONVERSATION TRACKING

In [ ]:
class ConversationMemory:
    """Enhanced memory system with complete conversation history and smart filtering"""

    def __init__(self):
        self.sessions = {}

    def get_session(self, session_id: str) -> Dict:
        """Get or create a session with enhanced fields"""
        if session_id not in self.sessions:
            self.sessions[session_id] = {
                "user_name": "",
                "interests": [],
                "skills": [],
                "preferences": [],
                "education_level": "unknown",
                "conversation_history": [],  # Store complete conversation context
                "suggested_careers": [],
                "last_suggestions": [],
                "searched_topics": [],
                "favorite_careers": [],
                "full_conversation": [],  # Store all user messages and responses
                "key_facts": [],  # Important facts mentioned by user
                "goals": [],  # User's stated goals
                "concerns": [],  # User's concerns or challenges
                "timeline": "",  # When they plan to make decisions
                "location_preference": "",  # Where they want to work/study
                "financial_situation": ""  # Budget concerns, scholarship needs, etc.
            }
        return self.sessions[session_id]

    def add_conversation_turn(self, session_id: str, user_message: str, bot_response: str):
        """Store complete conversation turn"""
        session = self.get_session(session_id)
        turn = {
            "timestamp": str(datetime.now()),
            "user_message": user_message,
            "bot_response": bot_response
        }
        session["full_conversation"].append(turn)

        # Keep only last 20 turns to manage memory
        if len(session["full_conversation"]) > 20:
            session["full_conversation"] = session["full_conversation"][-20:]

    def add_search_topic(self, session_id: str, topic: str, query_type: str):
        """FIXED: Add the missing method to track search topics"""
        session = self.get_session(session_id)
        search_entry = f"{topic} ({query_type})"
        if search_entry not in session["searched_topics"]:
            session["searched_topics"].append(search_entry)

        # Keep only last 10 searches
        if len(session["searched_topics"]) > 10:
            session["searched_topics"] = session["searched_topics"][-10:]

    def update_session(self, session_id: str, updates: Dict):
        """Update session data with smart merging"""
        session = self.get_session(session_id)
        for key, value in updates.items():
            if key in ["interests", "skills", "preferences", "searched_topics", "favorite_careers", "key_facts", "goals", "concerns"] and isinstance(value, list):
                # Merge lists without duplicates
                existing = [item.lower() for item in session[key]]
                for item in value:
                    if item.lower() not in existing:
                        session[key].append(item)
            else:
                session[key] = value

memory = ConversationMemory()


Searching Mechanism With Smart Filtering

In [ ]:
def intelligent_result_filtering(results: str, user_query: str, search_topic: str) -> str:
    """AI-powered intelligent filtering that adapts to any query without hardcoded rules"""

    if not results or not search_topic:
        return results

    # Extract meaningful keywords from user query and search topic
    import re

    def extract_key_terms(text: str) -> List[str]:
        """Extract meaningful terms from text"""
        # Remove common stop words
        stop_words = {'the', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'what', 'where', 'when', 'how', 'why', 'about', 'tell', 'me', 'i', 'you', 'my', 'your'}

        # Extract words longer than 2 characters
        words = re.findall(r'\b\w{3,}\b', text.lower())
        meaningful_words = [word for word in words if word not in stop_words]

        # Also extract phrases (2-3 words together)
        phrases = re.findall(r'\b\w+\s+\w+\b', text.lower())
        phrases += re.findall(r'\b\w+\s+\w+\s+\w+\b', text.lower())

        return meaningful_words + phrases

    # Get key terms from both user query and search topic
    query_terms = extract_key_terms(user_query)
    topic_terms = extract_key_terms(search_topic)
    all_key_terms = list(set(query_terms + topic_terms))

    if not all_key_terms:
        return results

    # Split results into sections
    sections = results.split("**")
    scored_sections = []

    for i, section in enumerate(sections):
        if i == 0:  # Always keep the header
            scored_sections.append((section, 1000))  # High score for header
            continue

        section_lower = section.lower()
        relevance_score = 0
        matched_terms = []

        # Score based on term matches
        for term in all_key_terms:
            if term in section_lower:
                # Higher score for exact matches in titles/URLs
                if term in section[:200].lower():  # First 200 chars (likely title/URL)
                    relevance_score += 10
                    matched_terms.append(term)
                else:
                    relevance_score += 5
                    matched_terms.append(term)

        # Bonus scoring for domain relevance
        domain_indicators = {
            'university': ['university', 'college', 'faculty', 'admission', 'course', 'program', 'degree', 'tuition', 'campus'],
            'career': ['job', 'career', 'salary', 'employment', 'work', 'profession', 'hiring', 'skills'],
            'company': ['company', 'corporation', 'business', 'employer', 'organization', 'firm']
        }

        for category, indicators in domain_indicators.items():
            for indicator in indicators:
                if indicator in section_lower:
                    relevance_score += 2

        # Store section with its score and matched terms
        scored_sections.append((section, relevance_score, matched_terms))

    # Sort by relevance score (descending)
    scored_sections.sort(key=lambda x: x[1] if len(x) > 1 else 0, reverse=True)

    # Determine filtering threshold
    if len(scored_sections) > 1:
        scores = [item[1] for item in scored_sections[1:]]  # Exclude header
        if scores:
            avg_score = sum(scores) / len(scores)
            threshold = max(5, avg_score * 0.3)  # Dynamic threshold
        else:
            threshold = 5
    else:
        threshold = 0

    # Filter sections based on relevance
    filtered_sections = []
    filtered_count = 0

    for item in scored_sections:
        section = item[0]
        score = item[1] if len(item) > 1 else 1000

        if score >= threshold or filtered_count == 0:  # Always keep header + relevant sections
            filtered_sections.append(section)
            if score < 1000:  # Don't count header
                filtered_count += 1

        if filtered_count >= 4:
            break

    # If we filtered out too much, be more lenient
    if filtered_count < 2 and len(scored_sections) > 3:
        filtered_sections = [item[0] for item in scored_sections[:5]]  # Take top 5 including header
        filtering_note = f"\n\n💡 *Showing broader results for '{search_topic}' - some may be partially relevant.*"
    else:
        filtering_note = f"\n\n✅ *Results intelligently filtered for: {search_topic}*"

    filtered_result = "**".join(filtered_sections)

    # Add filtering explanation
    if filtered_count > 0:
        filtered_result += filtering_note

    return filtered_result

DUAL SEARCH SYSTEM (Tavily + Google Search)

In [ ]:
def search_with_google(query: str) -> Optional[str]:
    if not GOOGLE_SEARCH_AVAILABLE:
        return None

    try:
        results = []
        for url in search(query, num_results=4):
            results.append(url)

        if results:
            return "📋 **Related Resources:**\n" + "\n".join([f"• {url}" for url in results])

    except Exception as e:
        print(f"Google search error: {e}")
        return None

def search_with_tavily(query: str, search_type: str = "general", user_query: str = "", search_topic: str = "") -> Optional[str]:
    """Enhanced search using Tavily API with smart filtering"""
    if not TAVILY_AVAILABLE:
        return None

    try:
        tavily_key = os.environ.get("TAVILY_API_KEY", "").strip()
        if not tavily_key or tavily_key == "YOUR_TAVILY_API_KEY_HERE":
            return None

        client = TavilyClient(api_key=tavily_key)

        # Enhanced search with more specific targeting
        response = client.search(
            query=query,
            search_depth="advanced",
            max_results=6,  # Get more results for better filtering
            include_domains=[
                "jobstreet.com.my", "linkedin.com", "glassdoor.com",
                "says.com", "studymalaysia.com", "afterschool.my",
                "qswur.com", "universitiesmalaysia.my", "um.edu.my",
                "utm.my", "upm.edu.my", "ukm.my", "usm.my"
            ]
        )

        if response.get("results"):
            formatted_results = ""
            for i, result in enumerate(response["results"][:4], 1):  # Show top 4
                title = result.get("title", "")
                content = result.get("content", "")
                url = result.get("url", "")

                # Extract key information based on search type
                if search_type == "salary" and any(word in content.lower() for word in ["rm", "ringgit", "salary", "pay"]):
                    content = content[:300] + "..."
                elif search_type == "university" and any(word in content.lower() for word in ["university", "college", "tuition", "fee"]):
                    content = content[:300] + "..."
                else:
                    content = content[:250] + "..."

                formatted_results += f"**{i}. {title}**\n{content}\n🔗 [Read more]({url})\n\n"

            # Use the intelligent filtering
            filtered_results = intelligent_result_filtering(formatted_results, user_query, search_topic)
            return filtered_results

    except Exception as e:
        print(f"Tavily search error: {e}")
        return None

def smart_career_search(topic: str, query_type: str, session_id: str, user_query: str = "") -> str:
    """Intelligent search with enhanced filtering - FIXED"""

    # Track what user is searching for - FIXED: Now calls the correct method
    memory.add_search_topic(session_id, topic, query_type)

    # Craft more specific queries based on type and topic
    queries = {
        "salary": f"{topic} average salary Malaysia 2024 jobstreet glassdoor",
        "tuition": f"{topic} university course tuition fees Malaysia 2024",
        "ranking": f"best {topic} universities Malaysia ranking QS Times",
        "companies": f"top companies hiring {topic} Malaysia multinational",
        "requirements": f"{topic} job requirements skills qualifications Malaysia",
        "prospects": f"{topic} career prospects job market Malaysia future",
        "pathway": f"how to become {topic} Malaysia education pathway steps",
        "general": f"{topic} career information Malaysia opportunities"
    }

    # Special handling for university-specific queries
    if any(uni in topic.lower() for uni in ["universiti malaya", "utm", "upm", "ukm", "usm", "um"]):
        queries["tuition"] = f"{topic} tuition fees courses programs Malaysia"
        queries["ranking"] = f"{topic} ranking programs courses Malaysia"
        queries["general"] = f"{topic} courses programs admission requirements Malaysia"

    search_query = queries.get(query_type, queries["general"])
    result_header = f"🔍 **{query_type.title()} Information for {topic}:**\n\n"

    # Try Tavily first with enhanced filtering
    tavily_result = search_with_tavily(search_query, query_type, user_query, topic)
    if tavily_result:
        return result_header + tavily_result

    # Fallback to Google search
    google_result = search_with_google(search_query)
    if google_result:
        return result_header + google_result

    # If both fail
    return f"🔍 I couldn't find current information about {topic} ({query_type}) right now. Please try again later."

# ENHANCED LLM LOGIC WITH COMPLETE MEMORY INTEGRATION

In [ ]:
def enhanced_career_analysis(user_input: str, session_id: str) -> Dict:
    """Enhanced analysis with complete conversation memory"""

    if not LLM_AVAILABLE:
        return {
            "is_career_related": True,
            "user_name": "",
            "new_interests": [],
            "new_skills": [],
            "new_preferences": [],
            "education_level": "unknown",
            "response": "I'm having trouble connecting to the AI service. Please check your API key.",
            "suggestions": [],
            "search_requests": [],
            "follow_up_question": None
        }

    session = memory.get_session(session_id)

    # Build comprehensive profile with complete conversation history
    recent_conversation = ""
    if session["full_conversation"]:
        recent_conversation = "\n".join([
            f"User: {turn['user_message']}\nBot: {turn['bot_response'][:200]}..."
            for turn in session["full_conversation"][-3:]  # Last 3 turns
        ])

    profile_summary = f"""
    User Name: {session['user_name'] or 'Unknown'}
    Interests: {session['interests']}
    Skills: {session['skills']}
    Preferences: {session['preferences']}
    Education Level: {session['education_level']}
    Goals: {session['goals']}
    Concerns: {session['concerns']}
    Key Facts: {session['key_facts']}
    Timeline: {session['timeline']}
    Location Preference: {session['location_preference']}
    Financial Situation: {session['financial_situation']}
    Previous Suggestions: {session['last_suggestions']}
    Previously Searched: {session['searched_topics']}
    Favorite Careers: {session['favorite_careers']}

    Recent Conversation Context:
    {recent_conversation}
    """

    # Enhanced prompt with complete context awareness
    prompt = f"""
You are GoalGuru, a smart and friendly AI career advisor for Malaysian users.
You have complete memory of the conversation and can understand context deeply.

User's current message: "{user_input}"

Complete User Profile & Context: {profile_summary}

Your enhanced tasks:

1. **Complete Context Understanding**: Use ALL conversation history to provide contextual responses.

2. **Name Recognition**: Extract name if mentioned.

3. **Enhanced Search Detection**: Detect if user wants specific information and identify the EXACT entity:
   - For universities: Extract the specific university name (e.g., "Universiti Malaya", "UTM")
   - For careers: Extract the specific career/field
   - For companies: Extract the specific company name
   - Query types: salary|tuition|ranking|companies|requirements|prospects|pathway|general

4. **Complete Memory Updates**: Extract ANY new information about the user:
   - Personal facts, family background, constraints
   - Goals, dreams, concerns, fears
   - Timeline preferences, location preferences
   - Financial situation, scholarship needs
   - Academic performance, extracurricular activities

5. **Contextual Career Guidance**: Use complete conversation history for better suggestions.

Respond in this JSON format:
{{
  "is_career_related": true or false,
  "user_name": "extracted name or empty string",
  "new_interests": ["any new interests mentioned"],
  "new_skills": ["any new skills mentioned"],
  "new_preferences": ["any new preferences mentioned"],
  "new_goals": ["any goals or aspirations mentioned"],
  "new_concerns": ["any concerns or challenges mentioned"],
  "new_key_facts": ["any important personal facts"],
  "timeline": "when they plan to make decisions",
  "location_preference": "where they want to work/study",
  "financial_situation": "budget concerns, scholarship needs",
  "education_level": "secondary|pre-university|undergraduate|graduate|working|unknown",
  "response": "Contextual response using conversation history and their name",
  "suggestions": [
    {{
      "career": "Career Name",
      "reason": "Why this fits based on complete profile and conversation",
      "path": {{
        "pre_university": "Program name",
        "university_course": "Course name",
        "example_universities": ["UM", "UTM", "UKM"],
        "scholarships": ["JPA", "MARA", "Yayasan Khazanah"]
      }},
      "next_steps": "Actionable steps based on their situation"
    }}
  ],
  "search_requests": [
    {{
      "topic": "EXACT entity name (e.g., 'Universiti Malaya', 'Data Scientist')",
      "query_type": "salary|tuition|ranking|companies|requirements|prospects|pathway|general"
    }}
  ],
  "follow_up_question": "Contextual question based on conversation history"
}}

IMPORTANT:
- For search requests, be very specific with the "topic" field
- Use complete conversation context to provide better responses
- Remember and reference previous parts of the conversation
"""

    try:
        response = llm_model.invoke([HumanMessage(content=prompt)])

        # Extract JSON
        match = re.search(r"{.*}", response.content, re.DOTALL)
        if match:
            parsed = json.loads(match.group())

            # Update memory with complete information
            updates = {}
            if parsed.get("user_name"):
                updates["user_name"] = parsed["user_name"]
            if parsed.get("new_interests"):
                updates["interests"] = session["interests"] + parsed["new_interests"]
            if parsed.get("new_skills"):
                updates["skills"] = session["skills"] + parsed["new_skills"]
            if parsed.get("new_preferences"):
                updates["preferences"] = session["preferences"] + parsed["new_preferences"]
            if parsed.get("new_goals"):
                updates["goals"] = session["goals"] + parsed["new_goals"]
            if parsed.get("new_concerns"):
                updates["concerns"] = session["concerns"] + parsed["new_concerns"]
            if parsed.get("new_key_facts"):
                updates["key_facts"] = session["key_facts"] + parsed["new_key_facts"]
            if parsed.get("timeline"):
                updates["timeline"] = parsed["timeline"]
            if parsed.get("location_preference"):
                updates["location_preference"] = parsed["location_preference"]
            if parsed.get("financial_situation"):
                updates["financial_situation"] = parsed["financial_situation"]
            if parsed.get("education_level") != "unknown":
                updates["education_level"] = parsed["education_level"]
            if parsed.get("suggestions"):
                updates["last_suggestions"] = [s["career"] for s in parsed["suggestions"]]
                updates["favorite_careers"] = session["favorite_careers"] + [s["career"] for s in parsed["suggestions"]]

            memory.update_session(session_id, updates)

            return parsed
        else:
            raise json.JSONDecodeError("No JSON found", response.content, 0)

    except Exception as e:
        print(f"❌ LLM error: {e}")
        return {
            "is_career_related": False,
            "user_name": "",
            "new_interests": [],
            "new_skills": [],
            "new_preferences": [],
            "education_level": "unknown",
            "response": "I'm here to help with your career questions! What would you like to know?",
            "suggestions": [],
            "search_requests": [],
            "follow_up_question": None
        }

ENHANCED CHATBOT WITH COMPLETE MEMORY AND SMART FILTERING

In [ ]:
def enhanced_chatbot_with_detailed_search(user_input: str, chat_history: List, session_id: str):
    """Enhanced chatbot with complete memory and smart search filtering - FIXED"""

    chat_history = chat_history or []
    session = memory.get_session(session_id)

    # Get AI analysis
    result = enhanced_career_analysis(user_input, session_id)

    # Build personalized response with conversation context
    user_name = session.get("user_name", "")
    greeting = f"Hi {user_name}! " if user_name else ""

    ai_response = result.get("response", "Thanks for sharing!")
    if greeting and not ai_response.startswith("Hi") and user_name:
        ai_response = greeting + ai_response

    suggestions = result.get("suggestions", [])
    search_requests = result.get("search_requests", [])

    # Handle search requests with enhanced filtering
    if search_requests:
        ai_response += "\n\n" + "="*50 + "\n"
        for search_req in search_requests:
            topic = search_req.get("topic", "")
            query_type = search_req.get("query_type", "general")

            if topic:
                # Pass the original user query for better filtering
                search_result = smart_career_search(topic, query_type, session_id, user_input)
                ai_response += f"\n{search_result}\n" + "="*50 + "\n"

    # Add career suggestions with complete context
    if suggestions:
        ai_response += f"\n\n🎯 **Career Suggestions{f' for {user_name}' if user_name else ''}:**\n"
        for i, suggestion in enumerate(suggestions, 1):
            ai_response += f"\n**{i}. {suggestion.get('career', 'Unknown')}**"
            ai_response += f"\n💡 {suggestion.get('reason', '')}"

            # Add detailed pathway
            path = suggestion.get("path", {})
            if path and any(path.values()):
                ai_response += f"\n\n📘 **Study Path:**"
                if path.get("pre_university"):
                    ai_response += f"\n• Pre-University: {path['pre_university']}"
                if path.get("university_course"):
                    ai_response += f"\n• University Course: {path['university_course']}"
                if path.get("example_universities"):
                    ai_response += f"\n• Universities: {', '.join(path['example_universities'])}"
                if path.get("scholarships"):
                    ai_response += f"\n• Scholarships: {', '.join(path['scholarships'])}"

            if suggestion.get("next_steps"):
                ai_response += f"\n🎯 **Next Steps:** {suggestion['next_steps']}"

            ai_response += "\n"

    # Add follow-up question
    follow_up = result.get("follow_up_question")
    if follow_up:
        ai_response += f"\n❓ {follow_up}"

    # Show enhanced memory context
    if any([session["interests"], session["skills"], user_name, session["goals"], session["key_facts"]]):
        ai_response += f"\n\n💭 **What I remember{f' about {user_name}' if user_name else ''}:**"
        if user_name:
            ai_response += f"\n• Name: {user_name}"
        if session["interests"]:
            ai_response += f"\n• Interests: {', '.join(session['interests'][-5:])}"  # Last 5
        if session["skills"]:
            ai_response += f"\n• Skills: {', '.join(session['skills'][-5:])}"
        if session["goals"]:
            ai_response += f"\n• Goals: {', '.join(session['goals'][-3:])}"
        if session["key_facts"]:
            ai_response += f"\n• Key Facts: {', '.join(session['key_facts'][-3:])}"
        if session["education_level"] != "unknown":
            ai_response += f"\n• Education: {session['education_level']}"
        if session["timeline"]:
            ai_response += f"\n• Timeline: {session['timeline']}"

    # Store complete conversation turn
    memory.add_conversation_turn(session_id, user_input, ai_response.strip())

    chat_history.append((user_input, ai_response.strip()))
    return chat_history, chat_history, ""

Build User Interface Using Gradio

In [ ]:
def initialize_enhanced_chat():
    """Initialize with enhanced greeting"""
    search_status = "✅ Tavily + Google" if TAVILY_AVAILABLE else "✅ Google Search" if GOOGLE_SEARCH_AVAILABLE else "❌ Search Disabled"

    initial_message = [
        (None, f"""👋 **Welcome to GoalGuru - Your Smart Career Advisor!**

🆕 **Enhanced Features:**
• **Personal Memory**: I remember your name and preferences
• **Detailed Search**: Ask about salaries, tuition fees, university rankings, top companies
• **Smart Recommendations**: Personalized career suggestions
• **Study Pathways**: Complete education roadmaps

🔍 **Search Status**: {search_status}

🎯 **Tell me:**
• Your name (I'll remember it!)
• Your interests, skills, education level
• Ask specific questions like:
  - "What's the salary for data scientists in Malaysia?"
  - "Which universities have the best engineering programs?"
  - "What are the tuition fees for computer science?"
  - "Which companies hire accountants?"

💡 **Example**: "Hi, I'm Sarah. I love math and I'm in Form 5. What's the salary for engineers?"

What would you like to explore today?""")
    ]
    return initial_message

# Build Enhanced Gradio Interface
print("🚀 Building enhanced Gradio interface...")

with gr.Blocks(title="GoalGuru - Enhanced Career Advisor", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎓 GoalGuru: Smart Career Advisor")
    gr.Markdown("*Enhanced with Personal Memory, Dual Search & Detailed Queries*")

    with gr.Row():
        session_input = gr.Textbox(
            label="🔑 Session ID",
            value="user_0001",
            info="Keep the same ID to maintain your personal profile and memory"
        )

    chatbot = gr.Chatbot(
        label="💬 Chat with GoalGuru",
        bubble_full_width=False,
        value=initialize_enhanced_chat(),
        height=600
    )

    state = gr.State([])

    with gr.Row():
        msg = gr.Textbox(
            placeholder="Introduce yourself, ask about careers, salaries, universities, or search for specific information...",
            scale=4,
            container=False,
            label="Your Message"
        )
        send_btn = gr.Button("Send", scale=1, variant="primary")

    # Event handlers
    send_btn.click(
        fn=enhanced_chatbot_with_detailed_search,
        inputs=[msg, state, session_input],
        outputs=[chatbot, state, msg]
    )

    msg.submit(
        fn=enhanced_chatbot_with_detailed_search,
        inputs=[msg, state, session_input],
        outputs=[chatbot, state, msg]
    )

    # Enhanced examples
    gr.Examples(
        examples=[
            ["Hi, I'm Alex. I love programming and I'm in Form 5"],
            ["What's the average salary for data scientists in Malaysia?"],
            ["Which universities have the best computer science programs?"],
            ["What are the tuition fees for engineering courses?"],
            ["Which companies hire marketing graduates?"],
            ["How do I become a doctor? What are the requirements?"],
            ["What's the job prospects for AI engineers?"]
        ],
        inputs=msg
    )

    # Status information
    with gr.Row():
        gr.Markdown(f"""
        **System Status:**
        - 🤖 LLM: {'✅ Connected' if LLM_AVAILABLE else '❌ Check API Key'}
        - 🔍 Tavily Search: {'✅ Available' if TAVILY_AVAILABLE else '⚠️ Not configured (optional)'}
        - 🌐 Google Search: {'✅ Available' if GOOGLE_SEARCH_AVAILABLE else '❌ Not available'}
        """)

🚀 Building enhanced Gradio interface...


/tmp/ipython-input-4177814379.py:45: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-4177814379.py:45: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


LAUNCH THE ENHANCED APP

In [ ]:
print("✅ Enhanced GoalGuru setup complete!")
print(f"🔍 Search capabilities: {'Tavily + Google' if TAVILY_AVAILABLE else 'Google only' if GOOGLE_SEARCH_AVAILABLE else 'Limited'}")

if __name__ == "__main__":
    demo.launch(share=True, debug=True)
else:
    demo.launch(share=True, debug=True)

✅ Enhanced GoalGuru setup complete!
🔍 Search capabilities: Tavily + Google
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f79d6c9bd4d2a219ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
